In [0]:

# 02_bronze_to_silver (Databricks notebook)

from pyspark.sql.window import Window
from pyspark.sql.functions import col, to_date, lag

# 👉 Set your catalog & schema; from your screenshot it looked like 'workspace' and 'default'
CATALOG = "workspace"
SCHEMA  = "default"

# 👉 Use the Bronze table name you already created
# If you used upload UI: it might be 'workspace.default.s_p_500_stock_prices_2014_2017'
# If you created your own bronze: 'workspace.default.stocks_bronze'
BRONZE_TBL = f"{CATALOG}.{SCHEMA}.s_p_500_stock_prices_2014_2017"  # adjust if needed

# 👉 Target Silver table (managed Delta table)
SILVER_TBL = f"{CATALOG}.{SCHEMA}.stocks_silver"

# ---- Read Bronze managed table (NOT a path) ----
df_bronze = spark.table(BRONZE_TBL)

# ---- Clean types & add returns ----
w = Window.partitionBy("symbol").orderBy(col("date"))

df_silver = (
    df_bronze
      .withColumn("date", to_date(col("date")))   # ensure proper date type
      .withColumn("open",  col("open").cast("double"))
      .withColumn("high",  col("high").cast("double"))
      .withColumn("low",   col("low").cast("double"))
      .withColumn("close", col("close").cast("double"))
      .withColumn("volume", col("volume").cast("long"))
      .withColumn("prev_close", lag(col("close")).over(w))
      .withColumn("return", (col("close") - col("prev_close"))/col("prev_close"))
      .drop("prev_close")
)

# ---- Write Silver as a managed table (NOT a path) ----
(df_silver.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(SILVER_TBL))

# ---- Verify ----
display(spark.table(SILVER_TBL))


symbol,date,open,high,low,close,volume,return
A,2014-01-02,57.1,57.1,56.15,56.21,1916160,null
A,2014-01-03,56.39,57.345,56.26,56.92,1866651,0.012631204412026345
A,2014-01-06,57.4,57.7,56.56,56.64,1777472,-0.004919184820801144
A,2014-01-07,56.95,57.63,56.93,57.45,1463208,0.014300847457627159
A,2014-01-08,57.33,58.54,57.17,58.39,2659468,0.016362053959965147
A,2014-01-09,58.4,58.68,57.87,58.41,1757647,3.425244048637784E-4
A,2014-01-10,58.51,59.01,58.14,58.93,1623330,0.008902585173771668
A,2014-01-13,58.77,59.1,58.6,58.93,2946738,0.0
A,2014-01-14,59.16,59.95,59.02,59.88,2562236,0.016120821313422753
A,2014-01-15,59.82,60.365,59.585,60.34,2335194,0.007682030728122927
